In [65]:
from linkml_runtime.utils.schemaview import SchemaView
mixs = '/data/projects/mixs/model/schema/mixs.yaml'

In [66]:
view = SchemaView(mixs)
soil_package = view.get_class('soil')


In [94]:
from yaml import load, dump
from yaml import CLoader as Loader, CDumper as Dumper
#except ImportError:
#    from yaml import Loader, Dumper
    
mixs_package = view.get_class('soil')
package_slot_usage = mixs_package.slot_usage
slotdef = package_slot_usage.get('samp_name','')

print(dump(slotdef, Dumper=Dumper))



!!python/object:linkml_runtime.linkml_model.meta.SlotDefinition
abstract: null
alias: null
aliases: []
all_members: {}
all_of: []
alt_descriptions: {}
annotations: {}
any_of: []
apply_to: []
asymmetric: null
broad_mappings: []
children_are_mutually_disjoint: null
close_mappings: []
comments: []
conforms_to: null
created_by: null
created_on: null
definition_uri: null
deprecated: null
deprecated_element_has_exact_replacement: null
deprecated_element_has_possible_replacement: null
description: null
designates_type: null
disjoint_with: []
domain: null
domain_of: []
equals_expression: null
equals_number: null
equals_string: null
equals_string_in: []
exact_mappings: []
exactly_one_of: []
examples: []
extensions: {}
from_schema: null
has_member: null
id_prefixes: []
identifier: null
ifabsent: null
imported_from: null
in_language: null
in_subset: []
inherited: null
inlined: null
inlined_as_list: null
inverse: null
irreflexive: null
is_a: null
is_class_field: null
is_grouping_slot: null
is_usag

In [95]:
slotdef = view.induced_slot('samp_name','soil')
print(dump(slotdef, Dumper=Dumper))

!!python/object:linkml_runtime.linkml_model.meta.SlotDefinition
abstract: null
alias: samp_name
aliases: []
all_members: !!python/object:jsonasobj2._jsonobj.JsonObj
  _if_missing: &id001 !!python/name:jsonasobj2._jsonobj._if_missing
all_of: []
alt_descriptions: !!python/object:jsonasobj2._jsonobj.JsonObj
  _if_missing: *id001
annotations: !!python/object:jsonasobj2._jsonobj.JsonObj
  _if_missing: *id001
any_of: []
apply_to: []
asymmetric: null
broad_mappings: []
children_are_mutually_disjoint: null
close_mappings: []
comments: []
conforms_to: null
created_by: null
created_on: null
definition_uri: null
deprecated: null
deprecated_element_has_exact_replacement: null
deprecated_element_has_possible_replacement: null
description: !!python/object/new:linkml_runtime.utils.yamlutils.extended_str
  args:
  - A local identifier or name that for the material sample used for extracting nucleic
    acids, and subsequent sequencing. It can refer either to the original material
    collected or to a

# setup dummy django project 
## MIXS_SCHEMA points to github checkout of https://github.com/GenomicsStandardsConsortium/mixs

In [80]:
import os, sys
import django
from django.template import Engine, Context, Template
from django.template.loader import get_template

PWD = os.getenv('PWD')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'django_mixs.settings')
sys.path.insert(0, '/home/vdejager/notebooks/mixs/django_mixs')

django.setup()



### dynamic forms & rendering based on
- https://simpleisbetterthancomplex.com/tutorial/2018/08/13/how-to-use-bootstrap-4-forms-with-django.html
- https://forum.djangoproject.com/t/django-templates-in-jupyter-avoid-kernel-restarts/11534
- https://forum.djangoproject.com/t/double-curly-braces-replacement/3584

I'm using crispy forms here for form formatting

In [81]:
from django import forms
from django.conf import settings
from linkml_runtime.utils.schemaview import SchemaView

from crispy_forms.helper import FormHelper
from crispy_forms.layout import Layout, Fieldset, ButtonHolder, Submit

# from crispy_forms.utils import render_crispy_form


class MIXSForm(forms.Form):
    MIXS_SCHEMA = settings.MIXS_SCHEMA

    def __init__(self, *args, **kwargs):
        if not 'package' in kwargs:
            raise TypeError( "must provide package= parameter value containing a valid MIXS6 package" )
        else:
            self.package = kwargs.pop('package')
        
        super(MIXSForm, self).__init__(*args, **kwargs)
        # initiate the cripsy form helper
        self.helper = FormHelper()
        self.helper.form_id = 'id-mixsForm'
        self.helper.form_class = 'blueForms'
        self.helper.form_method = 'post'
        self.helper.form_action = 'submit Mixs info'
        self.helper.add_input(Submit('submit', 'Submit'))
        self.helper.form_class = 'form-inline'
                
        # get the package information
        self._get_mixs_package(self.package)
        
        # create the fields based on slot usagee
        self._create_fields()
        
    def _create_fields(self):
        # get the slot usage for the package
        package_slot_usage = self.mixs_package.slot_usage
        
        # create the fields from the slot usage keys
        for field_name in package_slot_usage.keys():
            
            
            
            # https://docs.djangoproject.com/en/4.0/ref/forms/widgets/
            
            #default form widget for now
            
            self.fields[field_name] = self._build_field(field_name)
    
    
    def _build_field(self,field_name):
        
        # use the induced slot function here to get the definitions from terms.yaml 
        # since the slot definition from slot_usage does contain all the values
        # however this is slow
        # slotdef = package_slot_usage.get(field_name,'')
        slotdef = self.view.induced_slot(field_name, self.package)
        
        # add field widgets based on schema definition (in progress)
        if slotdef.range == 'integer':
            field_widget = forms.NumberInput
        if slotdef.range == 'integer':
            field_widget = forms.NumberInput
        else:
            field_widget = forms.TextInput
            
        fielddef = forms.CharField(widget = field_widget,
                                               required = slotdef.required,
                                               label = slotdef.title,
                                               help_text = slotdef.description,
                                              )
        return fielddef
    
    
    def _get_mixs_package(self, package):
        view = SchemaView(self.MIXS_SCHEMA)
        mixs_package = view.get_class(package)
        self.view = view
        self.mixs_package = mixs_package
        
        
    def clean(self):
        # cleanup data based on MIXS definitions, maybe we can used the dataclasses automatically made my the linkml runtime package
        pass

    def save(self):
        # save data 
        pass
    
    

In [85]:
form = MIXSForm(package='soil')

In [86]:
context = {
    'form' : form,
    'csrf_token': 'dodge UserWarning',
    'package' : form.package
}

t = get_template('mixsform.html')


In [87]:
from IPython.core.display import HTML
HTML(t.render(context))